### 확증적 데이터 분석(Confirmatory Data Analysis)
1. 목표변수 : 입원기간
2. 설명변수
    - 연속형 : BMI, 연령
    - 범주형 : 고혈압여부, 당뇨여부, 심혈관질환

In [3]:
import matplotlib.pyplot as plt
# 한글처리
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd
import scipy.stats as stats

In [4]:
df_ROS = pd.read_csv('../../dataset/RecurrenceOfSurgery_quest.csv')
df_ROS.head(2)

,Unnamed: 0,고혈압여부,당뇨여부,성별,스테로이드치료,신장,심혈관질환,연령,입원기간,체중,환자통증정도,수술시간,재발여부
0,0,0,0,2,1,163,0,66,2,60.3,10,68.0,0
1,1,0,0,1,1,171,0,47,1,71.7,10,31.0,0


In [39]:
df_ROS_patient = df_ROS[['신장','체중','고혈압여부','당뇨여부','심혈관질환','연령','입원기간']]
df_ROS_patient

,신장,체중,고혈압여부,당뇨여부,심혈관질환,연령,입원기간
0,163,60.3,0,0,0,66,2
1,171,71.7,0,0,0,47,1
2,178,77.1,0,0,0,39,1
3,174,74.2,1,0,0,40,1
4,183,80.7,1,0,0,42,2
...,...,...,...,...,...,...,...
1889,157,64.0,0,0,0,59,2
1890,157,59.0,0,0,0,42,4
1891,167,70.0,0,0,0,61,3
1892,177,77.0,0,0,0,29,4


In [6]:
# 데이터 타입 변환(int -> str)
df_ROS_patient[['고혈압여부','당뇨여부','심혈관질환']] = df_ROS_patient[['고혈압여부','당뇨여부','심혈관질환']].astype(str)
df_ROS_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   신장      1894 non-null   int64  
 1   체중      1894 non-null   float64
 2   고혈압여부   1894 non-null   object 
 3   당뇨여부    1894 non-null   object 
 4   심혈관질환   1894 non-null   object 
 5   연령      1894 non-null   int64  
 6   입원기간    1894 non-null   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 103.7+ KB


C:\Users\koreavc\AppData\Local\Temp\ipykernel_9392\668822120.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ROS_patient[['고혈압여부','당뇨여부','심혈관질환']] = df_ROS_patient[['고혈압여부','당뇨여부','심혈관질환']].astype(str)


#### 연속형/연속형 확인 (목표변수:입원기간/설명변수: BMI, 연령) - 비정규분포
- 신장과 체중은 서로 관계가 있는 변수들로 판단됨
- BMI(신장과 체중의 비율을 사용한 체중의 객관적인 지수) 컬럼을 추가함.

##### 선정 변수 : BMI, 입원기간
- 분석 내용 : BMI에 따라서 입원기간에 변화가 있는가?
    - 귀무가설: BMI에 따라서 입원기간에 변화가 없다.
    - 대립가설: BMI에 따라서 입원기간에 변화가 있다.
- 분석 결론 (기준 - P.value 0.05)
    - 통계 결론: pvalue=0.39, 귀무가설 참
    - 사실 결론: BMI에 따라서 입원기간에 변화가 없다.

In [7]:
df_ROS_patient.describe()

,신장,체중,연령,입원기간
count,1894.000000,1894.000000,1894.000000,1894.000000
mean,168.027983,68.111140,41.463569,1.736536
std,9.211419,11.234318,13.444629,2.344085
min,140.000000,35.000000,15.000000,0.000000
25%,161.000000,59.700000,32.000000,1.000000
50%,169.000000,68.000000,41.000000,1.000000
75%,175.000000,75.000000,49.000000,2.000000
max,204.000000,136.100000,86.000000,51.000000


In [8]:
# 목표 변수인 입원기간의 정규성 확인 -> 결과: pvalue=0.0, 비정규화
stats.shapiro(df_ROS_patient['입원기간'])

ShapiroResult(statistic=0.2543339729309082, pvalue=0.0)

In [9]:
# BMI 구하기 (BMI = 체중 / 신장(m)의 제곱)
def calculation (df_ROS_patient) :
    weight = df_ROS_patient['체중']
    height = df_ROS_patient['신장'] * 0.01
    BMI = weight / (height * height)
    return BMI

In [10]:
df_ROS_patient['BMI'] = df_ROS_patient.apply(calculation, axis=1)
df_ROS_patient['BMI']

C:\Users\koreavc\AppData\Local\Temp\ipykernel_9392\3258908706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ROS_patient['BMI'] = df_ROS_patient.apply(calculation, axis=1)


0       22.695623
1       24.520365
2       24.334049
3       24.507861
4       24.097465
          ...    
1889    25.964542
1890    23.936062
1891    25.099502
1892    24.577867
1893    17.361111
Name: BMI, Length: 1894, dtype: float64

In [13]:
stats.spearmanr(df_ROS_patient['BMI'], df_ROS_patient['입원기간'])

SignificanceResult(statistic=0.01961205375717151, pvalue=0.39363861617868634)

##### 선정 변수 : 연령, 입원기간
- 분석 내용 : 연령에 따라서 입원기간에 변화가 있는가?
    - 귀무가설: 연령에 따라서 입원기간에 변화가 없다.
    - 대립가설: 연령에 따라서 입원기간에 변화가 있다.
- 분석 결론 (기준 - P.value 0.05)
    - 통계 결론: pvalue=0.002, 대립가설 참
    - 사실 결론: 연령에 따라서 입원기간에 변화가 있다.

In [16]:
df_ROS_patient[['연령','입원기간']].describe()  # count값 동일함

,연령,입원기간
count,1894.000000,1894.000000
mean,41.463569,1.736536
std,13.444629,2.344085
min,15.000000,0.000000
25%,32.000000,1.000000
50%,41.000000,1.000000
75%,49.000000,2.000000
max,86.000000,51.000000


In [17]:
df_age_date = df_ROS_patient[['연령','입원기간']]
df_age_date[:2]

,연령,입원기간
0,66,2
1,47,1


In [18]:
df_age_date.corr()
# 연령과 입원기간의 상관계수 : 0.05
# 두 개의 변수는 상관관계 없음

,연령,입원기간
연령,1.000000,0.050292
입원기간,0.050292,1.000000


In [19]:
stats.spearmanr(df_ROS_patient['연령'], df_ROS_patient['입원기간'])

SignificanceResult(statistic=0.06940175801675035, pvalue=0.0025109193902937083)

#### 연속형/범주형 확인 (목표변수:입원기간/설명변수:고혈압여부, 당뇨여부, 심혈관질환)

##### 선정 변수 : 고혈압여부, 입원기간
- 분석 내용 : 고혈압여부에 따라서 입원기간에 변화가 있는가?
    - 귀무가설: 고혈압여부에 따라서 입원기간에 변화가 없다.
    - 대립가설: 고혈압여부에 따라서 입원기간에 변화가 있다.
- 분석 결론 (기준 - P.value 0.05)
    - 통계 결론: pvalue=1.468698026856349e-142, 대립가설 참
    - 사실 결론: 고혈압여부에 따라서 입원기간에 변화가 있다.

In [33]:
df_ROS_BP = df_ROS_patient[['고혈압여부','입원기간']]
df_ROS_BP[:2]

,고혈압여부,입원기간
0,0,2
1,0,1


In [40]:
# 고혈압여부 항목 2개(0,1)
df_BP_zero = df_ROS_BP[df_ROS_BP['고혈압여부'] == '0']
df_BP_zero[:2]

,고혈압여부,입원기간
0,0,2
1,0,1


In [54]:
df_BP_first = df_ROS_BP[df_ROS_BP['고혈압여부'] == '1']
df_BP_first[:2]

,고혈압여부,입원기간
3,1,1
4,1,2


In [44]:
# 검증
stats.ranksums(df_BP_zero['고혈압여부'], df_BP_first['고혈압여부'])

RanksumsResult(statistic=-25.42124264933584, pvalue=1.468698026856349e-142)

##### 선정 변수 : 당뇨여부, 입원기간
- 분석 내용 : 당뇨여부에 따라서 입원기간에 변화가 있는가?
    - 귀무가설: 당뇨여부에 따라서 입원기간에 변화가 없다.
    - 대립가설: 당뇨여부에 따라서 입원기간에 변화가 있다.
- 분석 결론 (기준 - P.value 0.05)
    - 통계 결론: pvalue=1.0616331899646055e-74, 대립가설 참
    - 사실 결론: 당뇨여부에 따라서 입원기간에 변화가 있다.

In [29]:
# 당뇨여부 항목 2개(0,1)
df_ROS_patient['당뇨여부'].value_counts()

0    1775
1     119
Name: 당뇨여부, dtype: int64

In [45]:
df_ROS_DM = df_ROS_patient[['당뇨여부','입원기간']]
df_ROS_DM[:2]

,당뇨여부,입원기간
0,0,2
1,0,1


In [57]:
# 당뇨여부 항목 2개(0,1)
df_DM_zero = df_ROS_DM[df_ROS_DM['당뇨여부'] == 0]
df_DM_zero[:2]

,당뇨여부,입원기간
0,0,2
1,0,1


In [58]:
df_DM_first = df_ROS_DM[df_ROS_DM['당뇨여부'] == 1]
df_DM_first[:2]

,당뇨여부,입원기간
6,1,1
75,1,1


In [59]:
# 검증
stats.ranksums(df_DM_zero['당뇨여부'], df_DM_first['당뇨여부'])

RanksumsResult(statistic=-18.286419546802723, pvalue=1.0616331899646055e-74)

##### 선정 변수 : 심혈관질환여부, 입원기간
- 분석 내용 : 심혈관질환여부에 따라서 입원기간에 변화가 있는가?
    - 귀무가설: 심혈관질환여부에 따라서 입원기간에 변화가 없다.
    - 대립가설: 심혈관질환여부에 따라서 입원기간에 변화가 있다.
- 분석 결론 (기준 - P.value 0.05)
    - 통계 결론: 
    - 사실 결론: 

In [30]:
# 심혈관질환 항목 2개(0,1)
df_ROS_patient['심혈관질환'].value_counts()

0    1826
1      68
Name: 심혈관질환, dtype: int64

In [60]:
df_ROS_CVD = df_ROS_patient[['심혈관질환','입원기간']]
df_ROS_CVD[:2]

,심혈관질환,입원기간
0,0,2
1,0,1


In [65]:
# 심혈관질환 항목 2개(0,1)
df_CVD_zero = df_ROS_CVD[df_ROS_CVD['심혈관질환'] == 0]
df_CVD_zero[:2]

,심혈관질환,입원기간
0,0,2
1,0,1


In [62]:
df_CVD_first = df_ROS_CVD[df_ROS_CVD['심혈관질환'] == 1]
df_CVD_first[:2]

,심혈관질환,입원기간
23,1,1
43,1,2


In [64]:
# 검증
stats.ranksums(df_CVD_zero['심혈관질환'], df_CVD_first['심혈관질환'])

NameError: name 'df_CVD_first' is not defined